### Reliable-RAG

#### Overview

###### The "Reliable-RAG" method enhances the traditional Retrieval-Augmented Generation (RAG) approach by adding layers of validation & refinement to ensure the accuracy & relevance of retrieved information. This system is designed to process & query web-based documents, encode their content with a vector store, and retrive the most relevant segments for generating precise and reliable answers. The method incorporates checks for document relevancy, hallucination prevention, and highlights the exact segments used in generating the final response

#### Key Components

##### 1. Document Loading & Checking:
###### Web-based documents are loaded and split into smaller, manageable chunks to facilitate efficient vector encoding & retrieval.

##### 2. Vectorstore Creation:
###### Utilizes Chroma and cohere embeddings to encode document chunks into a vector store, enabling efficient similarity-based retrieval.

##### 3. Document Relevancy Check:
###### Implements a relevance-checking mechanism using a language model to filter out non-relevant documents before answer generation

##### 4. Answer Generation:
###### Employs a language model to generate concise answers  based on the relevant documents retrieved.

##### 5. Hallucination Detection
###### A dedicated hallucination detection step ensures that the generated answers are grounded in the retrieval documents, preventing the inclusion of  unsupported or erroneous information.

##### 6. Document Snippet Highlighting:
###### The system identifies & highlights the specific document segments that were directly used to generate the answer, providing transparency & traceability


#### Motivation

###### The reliable-RAG m